In [20]:
import pandas as pd

data: str = '../../data/raw/wordle_withcritic_new_raw.jsonl'
save_path: str = '../../data/processed/wordle_withcritic_new_processed.jsonl'
df = pd.read_json(data, lines=True)

In [21]:
columns_to_keep_p1: list = ['game', 'benchmark_version', 'game_id', 'model', 'experiment', 'episode', 'Aborted', 'Lose', 'Success', 'chat_p1', 'target']

columns_to_keep_p2: list = ['game', 'benchmark_version', 'game_id', 'model', 'experiment', 'episode', 'Aborted', 'Lose', 'Success', 'chat_p2', 'target']

df['target'] = df.target_word

In [22]:
# Create a new DataFrame with duplicated entries
df_player1 = df[columns_to_keep_p1].rename(columns={'chat_p1': 'chat'})
df_player1['player'] = 'player 1'

df_player2 = df[columns_to_keep_p2].rename(columns={'chat_p2': 'chat'})
df_player2['player'] = 'player 2'

# Concatenate both DataFrames
result_df = pd.concat([df_player1, df_player2], ignore_index=True)

# Show the resulting 
len(result_df)

7582

## Filter all conversations that are successful and have an invalid format inside the assistant text

In [23]:
# Define the text to filter out
specific_text = 'INVALID_FORMAT'

# Function to check if specific_text is in any of the dictionaries in the list
def contains_specific_text(row):
      return any(specific_text in d['content'] for d in row)

In [24]:
# Filter the DataFrame
filtered_data = result_df[~result_df['chat'].apply(contains_specific_text)]

In [25]:
len(result_df), len(filtered_data)

(7582, 7448)

## Finding:
Wordle With clue and critic has a lot of INVALID_FORMAT in its explanations. When this is used to train the model to deliver content, 
it will learn to produce only the INVALID_CONTENT token. This does not help the game to be better played

In [26]:
filtered_data.to_json(save_path, orient='records', lines=True)

In [27]:
filtered_data.columns

Index(['game', 'benchmark_version', 'game_id', 'model', 'experiment',
       'episode', 'Aborted', 'Lose', 'Success', 'chat', 'target', 'player'],
      dtype='object')